In [1]:
import requests
import pandas as pd
import numpy as np
import datetime,time,random

In [2]:
class GenerateSinaAPIUrl():
    # this is to generate the url of SinaAPI
    
    def __init__(self,generUrl='http://hq.sinajs.cn/list='):
        self.generUrl=generUrl
        pass
    def getGreekIndicatorsurl(self,OptionCode):
        #this function is for get url of the greek indicators
        
        
        if isinstance(OptionCode,str):
            if 'CON_OP_' in OptionCode:
                sfGreek=OptionCode.replace('CON_OP_','CON_SO_')
        
                return self.generUrl+sfGreek
            else:
                raise ValueError('Unknown OptionCode:%s'%OptionCode)
        elif isinstance(OptionCode,(list,set,tuple)):
            OptionCodestr=','.join(OptionCode)
            if 'CON_OP_' in OptionCode:
                sfGreek=OptionCode.replace('CON_OP_','CON_SO_')
                return self.generUrl+sfGreek
            else:
                raise ValueError('Unknown OptionCode:%s'%OptionCodestr)  
    def getCodeofOptionurl(self,TargetAssetsCode,yearmonthcode,OptionType='Call',returnparameters=False):
        # this function is to get the api of  the  optioncode list 
        #getCodeofCallOptionurl='http://hq.sinajs.cn/list=OP_UP_%d%s'%(TargetAssetsCode,yearmonthcode)#获得某月到期的看涨期权代码列表
        #getCodeofPutOptionurl='http://hq.sinajs.cn/list=OP_DOWN_%d%s'%(TargetAssetsCode,yearmonthcode)#获得某月到期的看跌期权代码列表
        if isinstance(yearmonthcode,str):
            if OptionType=='Call':
                if returnparameters:
                    #http://hq.sinajs.cn/list=
                    return self.generUrl+'OP_UP_%d%s'%(TargetAssetsCode,yearmonthcode),'OP_UP_%d%s'%(TargetAssetsCode,yearmonthcode)
                else:
                    #http://hq.sinajs.cn/list=
                    return self.generUrl+'OP_UP_%d%s'%(TargetAssetsCode,yearmonthcode)
            elif OptionType=='Put':

                if returnparameters:
                    #http://hq.sinajs.cn/list=
                    return self.generUrl+'OP_DOWN_%d%s'%(TargetAssetsCode,yearmonthcode),'OP_DOWN_%d%s'%(TargetAssetsCode,yearmonthcode)
                else:
                    #http://hq.sinajs.cn/list=
                    return self.generUrl+'OP_DOWN_%d%s'%(TargetAssetsCode,yearmonthcode)
        elif isinstance(yearmonthcode,(set,tuple,list)):
            if OptionType=='Call':
                if returnparameters:
                    c=['OP_UP_%d%s'%(TargetAssetsCode,ymc) for ymc in yearmonthcode]
                    #http://hq.sinajs.cn/list=
                    r=self.generUrl+','.join(c)
                    return r,c
                else:
                    #http://hq.sinajs.cn/list=
                    return self.generUrl+','.join(['OP_UP_%d%s'%(TargetAssetsCode,ymc) for ymc in yearmonthcode])
            elif OptionType=='Put':
                if returnparameters:
                    c=['OP_DOWN_%d%s'%(TargetAssetsCode,ymc) for ymc in yearmonthcode]
                    #http://hq.sinajs.cn/list=
                    r=self.generUrl++','.join(c)
                    return r,c
                else:
                    #http://hq.sinajs.cn/list=
                    return self.generUrl++','.join(['OP_DOWN_%d%s'%(TargetAssetsCode,ymc) for ymc in yearmonthcode])
            
    def getRealtimeOptionQuotesurl(self,OptionCode):
        # this functin is to obtain the realtime option contract quote
        #print 'http://hq.sinajs.cn/list=%s'%(OptionCode)
        return 'http://hq.sinajs.cn/list=%s'%(OptionCode)#根据合约代码获得实时期权行情
    def getMonthofContracturl(self):
        return 'http://stock.finance.sina.com.cn/futures/api/openapi.php/StockOptionService.getStockName'#获得现在有哪几个月份的合约

class SinaAPI():
    def __init__(self,parentClass=GenerateSinaAPIUrl()):
        self.GenerateSinaAPIUrl=parentClass
        import requests
        self.session=requests.session()
        
    def get(self,url):
        response=self.session.get(url)
        if response.status_code==200:
            return response.text
        else:
            raise ValueError('Requests Error Code:%d'%response.status_code)
    def getMonth(self):
        requesttext=eval(self.get(self.GenerateSinaAPIUrl.getMonthofContracturl()))
        cateId=requesttext['result']['data']['cateId']
        contractMonth=requesttext['result']['data']['contractMonth']
        return cateId,contractMonth
    def getRealtimeOptionQuotes(self,OptionCode):
        RealtimeOptionQuotesurl=self.GenerateSinaAPIUrl.getRealtimeOptionQuotesurl(OptionCode)
        self.get(RealtimeOptionQuotesurl)
    def getCodeofOption(self,TargetAssetsCode,yearmonthcode,OptionType='Call'):
        
        CodeofOptionurl,para=self.GenerateSinaAPIUrl.getCodeofOptionurl(TargetAssetsCode,yearmonthcode,OptionType=OptionType,returnparameters=True)
        calltext=self.get(CodeofOptionurl)
        calltext.split(';\n')[:-1]
        #header='var hq_str_%s='%(para)
        for ctext in calltext.split(';\n')[:-1]:
            if 'var hq_str_' in ctext:
                code=ctext.split('=')[-1].split(',')
                calllist=[c.strip('"') for c in code if 'CON_OP_' in c]
                return calllist
            else:
                raise ValueError('Unknown Response Text of ')
    def getGreekIndicators(self,OptionCode):
        RequestTimeStamp=time.time()
        dgvt=self.get('http://hq.sinajs.cn/list='+','.join(OptionCode).replace('CON_OP_','CON_SO_'))
        colname=u'期权合约简称,,,,成交量,Delta,Gamma,Theta,Vega,隐含波动率,最高价,最低价,交易代码,行权价,最新价,理论价值,Unknown Var'.split(',')
        df=pd.DataFrame([item.split('=')[-1].strip('"').split(',')   for item in dgvt.split(';\n')[:-1]],columns=colname)
        df['RequestTimeStamp']=RequestTimeStamp
        df=df[[u'期权合约简称',  u'成交量',            u'Delta',
                  u'Gamma',            u'Theta',             u'Vega',
                  u'隐含波动率',              u'最高价',              u'最低价',
                   u'交易代码',              u'行权价',              u'最新价',
                   u'理论价值',      u'Unknown Var', u'RequestTimeStamp']]
        df.columns=['OptionContShortName','TradingVolume','Delta','Gamma','Theta','Vega','IV','High','Low',
                   'TrasactionCode','StrikePrice','LastestPrice','TheoreticalValue','Unknown Var','RequestTimeStamp']
        return df
    def getRealtimeOptionQuotes(self,OptionCode):
        """index=[u'买量',u'买价',u'最新价',u'卖价',u'卖量',
                   u'持仓量',u'涨幅',u'行权价',u'昨收价',u'开盘价',
                   u'涨停价',u'跌停价',u'申卖价五',u'申卖量五',u'申卖价四',
                   u'申卖量四',u'申卖价三',u'申卖量三',u'申卖价二',u'申卖量二',
                   u'申卖价一',u'申卖量一',u'申买价一',u'申买量一',u'申买价二',
                   u'申买量二',u'申买价三',u'申买量三',u'申买价四',u'申买量四',
                   u'申买价五',u'申买量五',u'行情时间',u'主力合约标识',u'状态码',
                   u'标的证券类型',u'标的股票',u'期权合约简称',u'振幅',u'最高价',
                   u'最低价',u'成交量',u'成交额'] """
        index=[u'BuyVolume',u'BuyPrice',u'LastestPrice',u'SellPrice',u'SellVol',
                   u'HoldingVolume',u'Change',u'StrikePrice',u'YesterdayClosePrice',u'OpenPrice',
                   u'HighPriceinTheory',u'LowPriceinTheory',u'SellPriceT5',u'SellVolumeT5',u'SellPriceT4',
                   u'SellVolumeT4',u'SellPriceT3',u'SellVolumeT3',u'SellPriceT2',u'SellVolumeT2',
                   u'SellPriceT1',u'SellVolumeT1',u'BuyPriceT1',u'BuyVolumeT1',u'BuyPriceT2',
                   u'BuyVolumeT2',u'BuyPriceT3',u'BuyVolumeT3',u'BuyPriceT4',u'BuyVolumeT4',
                   u'BuyPriceT5',u'BuyVolumeT5',u'QuotatesTime',u'MainContractCode',u'StatusCode',
                   u'UnderlyingSecCode',u'UnderlyingSec',u'OptionContShortName',u'Amplitude',u'MaxPriceToday',
                   u'MinPriceToday',u'TradingVolume',u'Amount']
        if isinstance(OptionCode,str):
            RealtimeOptionQuotesurl=self.GenerateSinaAPIUrl.getRealtimeOptionQuotesurl(OptionCode)
            #print RealtimeOptionQuotesurl
            RequestTimeStamp=time.time()
            stext=self.get(RealtimeOptionQuotesurl)
            header='var hq_str_%s='%(OptionCode)
            #print 1,stext,header
            data=stext.split(header)[-1].strip(';\n').strip('"').split(',')
            otherindex=[u'Unknown Var']*(len(data)-len(index))
            data.append(RequestTimeStamp)
            #print otherindex
            #print index+otherindex
            return pd.DataFrame(data,index=index+otherindex+['RequestTimeStamp'],columns=[OptionCode])
        elif isinstance(OptionCode,(list,set,tuple)):
            OptionCodestr=','.join(OptionCode)
            #print OptionCodestr
            RealtimeOptionQuotesurl=self.GenerateSinaAPIUrl.getRealtimeOptionQuotesurl(OptionCodestr)
            RequestTimeStamp=time.time()
            stext=self.get(RealtimeOptionQuotesurl)
            headers=['var hq_str_%s='%(s) for s in OptionCode]
            #data.append(RequestTimeStamp)
            c=dict()
            #print stext
            for r in stext.split(';\n')[:-1]:
                name,data=r.split('=')
                data=data.strip('"').split(',')
                #print data
                name=name.strip('var hq_str_')
                #otherindex=[u'Unknown Var']*(len(data)-len(index))
                c[name]=data
            tempdf= pd.DataFrame(c)
             
            otherindex=[u'Unknown Var']*(tempdf.shape[0]-len(index))
            tempdf.index=index+otherindex
            tempdf=tempdf.T
            tempdf['RequestTimeStamp']=RequestTimeStamp
            return tempdf
       
                


class Collector():
    def __init__(self,api=SinaAPI(GenerateSinaAPIUrl())):
        self.api=api
        self.watchlist=[]
        self.task=[]
        self.lastreqtime=0
    def startOnce(self,TargetAssetsCode=510050):
        cateId,contractMonth=self.api.getMonth()
        OptionContractMonth=sorted([d.split('-')[0][-2:]+d.split('-')[1] for d in set(contractMonth)])
        todayjobs={}
        for yearmonthcode in OptionContractMonth:
            calllist=self.api.getCodeofOption(TargetAssetsCode,yearmonthcode,OptionType='Call')
            putlist=self.api.getCodeofOption(TargetAssetsCode,yearmonthcode,OptionType='Put')
            todayjobs[yearmonthcode]={'Call':calllist,'Put':putlist}
        return todayjobs
        
        
    def beforeStart2(self,starttime,do_something,repeat=False,force=False):
        import datetime
        def checkdigit(i,digit=2):
            if len(str(i))>digit:
                return str(i)[-1*digit:]
            elif len(str(i))<=digit:
                return '0'*(-len(str(i))+digit)+str(i)
        
        n=datetime.datetime.now()
        d,h,m,s=n.day,n.hour,n.minute,n.second
        #str(h)+':'+str(m)>'12:23'
        if repeat !=False:
            if repeat=='Daily' and len(starttime)==5:
                if str(h)+':'+str(m)<=starttime or force ==True:
                    so_something
                else:
                    pass
    def beforeStart(self,TargetAssetsCode=510050):
        
        sssO=self.startOnce(TargetAssetsCode=TargetAssetsCode)
        self.watchlist=pd.DataFrame([(c,cp,k)  for k,v in sssO.items() for cp,v2 in v.items() for c in v2],columns=['SinaCode','OptionType','YM'])

        returnvalue=self.watchlist['SinaCode'].values
        self.watchlist=self.watchlist.set_index('SinaCode')
        
        return returnvalue
    def regularGetGI(self,watchlist,Optimizer=True):
        task=set(watchlist)
        
        df=self.api.getGreekIndicators(watchlist)
        
        return self.dataFrameOptimizer(df,Optimizer=Optimizer)
        
    def regularGetRTOQ(self,watchlist,repeatfreq=None,endtime=111111111111111111111111,merge=True,Optimizer=False):
        #self.task=set(watchlist)
        task=set(watchlist)
        if repeatfreq !=None:
            while time.time.now()>endtime+1:
                df=self.api.getRealtimeOptionQuotes(task)
                if merge:
                    return self.dataFrameOptimizer(pd.concat([df,self.watchlist],axis=1).reset_index(),Optimizer=Optimizer)
                else:
                    return self.dataFrameOptimizer(df,Optimizer=Optimizer)
        else:
            
            df=self.api.getRealtimeOptionQuotes(task)
            if merge:
                return self.dataFrameOptimizer(pd.concat([df,self.watchlist],axis=1).reset_index(),Optimizer=Optimizer)
            else:
                return self.dataFrameOptimizer(df,Optimizer=Optimizer)
    def dataFrameOptimizer(self,df2,Optimizer=True,outsiderule=False,**rules):
        if Optimizer:
            for v in df2.columns:
                try:
                    df2[v]=df2[v].astype(np.float)
                except ValueError as e:
                    pass
        else:
            pass
        for key,nptype in rules.items():
            if key in df2.columns:
                df2[key]=df[key].astype(nptype)
        return df2
        
        
            
class inMemoryDB():
    def __init__(self):
        import sqlite3
        self.conn=[]
        self.memory=True
    def createSqliteDB(self,path='data/',optionname='50ETF',typeofinfo='Trading',memory=False):
        import sqlite3
        n=datetime.datetime.now()
        if memory:
            dbname=':memory:'
        else:
            dbname=path+'Option-%s-%s-%s.sqlite'%(optionname,typeofinfo,n.strftime('%Y-%m-%d'))
        self.conn=sqlite3.connect(dbname)
        return self.conn
    def read(self,sql):
        return self.conn(sql).fetchall()
    def pdread(self,sql):
        return pd.read_sql(sql, self.conn)
        
def autoupdate(addmonth=0):
    import datetime
    FourdigitDate=datetime.datetime.today()
    year=str(FourdigitDate.year)[-2:]
    month='0'+str(FourdigitDate.month) if len(str(FourdigitDate.month))==1 else str(FourdigitDate.month)
    return year+month   
def timestampe2str(t=time.time()):
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(t))

def checkmem(var):
    #print sys.getsizeof(var)
    return sys.getsizeof(var)
       
                    
         
            
            
        

In [3]:
cc=Collector()       
TargetAssetsCode=510050
ss=cc.beforeStart()
iDB=inMemoryDB()
conn=iDB.createSqliteDB(memory=True)
#sinaapi=SinaAPI(GenerateSinaAPIUrl()) 
date=time.strftime('%Y-%m-%d',time.localtime(time.time()))
mstart=time.mktime(time.strptime(date+' 09:30:00', '%Y-%m-%d %H:%M:%S'))
mend=time.mktime(time.strptime(date+' 11:30:00', '%Y-%m-%d %H:%M:%S'))
afstart=time.mktime(time.strptime(date+' 13:00:00', '%Y-%m-%d %H:%M:%S'))
afend=time.mktime(time.strptime(date+' 15:00:00', '%Y-%m-%d %H:%M:%S'))
force=100

while True:
    if mend>= time.time()>=mstart or afstart<=time.time()<=afend:
        #start=timestampe2str()
        realtimeoption=cc.regularGetRTOQ(ss,Optimizer=True)
        #greek=cc.api.getGreekIndicators(ss)
        #cc.regularGet(ss)[u'行情时间'].max(),n.strftime(format='%Y-%m-%d %H:%M:%S')
        greek=cc.regularGetGI(ss,Optimizer=True)   
        #for i in xrange(110*2):
            #f=time.time()
        realtimeoption.to_sql('option',conn,if_exists='append')
        greek.to_sql('greek',conn,if_exists='append')
        time.sleep(1)
    elif mend<time.time()<afstart:
        time.sleep(afstart-time.time())
    elif force!=0:
        realtimeoption=cc.regularGetRTOQ(ss,Optimizer=True)
        #greek=cc.api.getGreekIndicators(ss)
        #cc.regularGet(ss)[u'行情时间'].max(),n.strftime(format='%Y-%m-%d %H:%M:%S')
        realtimegreek=cc.regularGetGI(ss,Optimizer=True)   
        #for i in xrange(110*2):
            #f=time.time()
        realtimeoption.to_sql('option',conn,if_exists='append')
        realtimegreek.to_sql('greek',conn,if_exists='append')
        time.sleep(random.random())
        force-=1
        print force
        
    else:
        option=iDB.pdread('select * from option')
        greek=iDB.pdread('select * from greek')
        #dbname=path+'Option-%s-%s-%s.sqlite'%(optionname,typeofinfo,n.strftime('%Y-%m-%d'))
        optionname='50ETF'
        n=datetime.datetime.now()
        option.to_hdf('data/Option-%s-%s-%s.h5'%(optionname,'Trading',n.strftime('%Y-%m-%d')),'RealtimeOPtion')
        greek.to_hdf('data/Option-%s-%s-%s.h5'%(optionname,'Greek',n.strftime('%Y-%m-%d')),'RealtimeGreek')
        conn.close()
        break




/Users/sn0wfree/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


/Users/sn0wfree/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block2_values] [items->['index', 'QuotatesTime', 'StatusCode', 'UnderlyingSecCode', 'OptionContShortName', 'Unknown Var', 'OptionType']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
/Users/sn0wfree/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block2_values] [items->['OptionContShortName', 'TrasactionCode', 'Unknown Var']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [4]:
pd.read_hdf('data/Option-50ETF-Trading-2018-04-20.h5')

,level_0,index,BuyVolume,BuyPrice,LastestPrice,SellPrice,SellVol,HoldingVolume,Change,StrikePrice,...,OptionContShortName,Amplitude,MaxPriceToday,MinPriceToday,TradingVolume,Amount,Unknown Var,RequestTimeStamp,OptionType,YM
0,0,CON_OP_10001025,8.0,0.1037,0.1033,0.1050,10.0,3783.0,-21.21,2.651,...,50ETF购6月2651A,20.90,0.1287,0.1013,942.0,1048695.47,A,1.524237e+09,Call,1806.0
1,1,CON_OP_10001026,8.0,0.0824,0.0836,0.0839,1.0,2459.0,-21.58,2.700,...,50ETF购6月2700A,21.20,0.1035,0.0809,241.0,219221.94,A,1.524237e+09,Call,1806.0
2,2,CON_OP_10001027,2.0,0.0657,0.0668,0.0665,1.0,2245.0,-22.33,2.749,...,50ETF购6月2749A,21.86,0.0834,0.0646,597.0,417755.09,A,1.524237e+09,Call,1806.0
3,3,CON_OP_10001028,1.0,0.0519,0.0523,0.0528,1.0,3065.0,-24.53,2.798,...,50ETF购6月2798A,24.53,0.0672,0.0502,618.0,330566.40,A,1.524237e+09,Call,1806.0
4,4,CON_OP_10001029,1.0,0.0417,0.0417,0.0419,10.0,4198.0,-24.73,2.847,...,50ETF购6月2847A,24.91,0.0540,0.0402,467.0,214356.57,A,1.524237e+09,Call,1806.0
5,5,CON_OP_10001030,11.0,0.1058,0.1073,0.1075,1.0,3546.0,19.35,2.651,...,50ETF沽6月2651A,26.03,0.1126,0.0892,1230.0,1242448.80,A,1.524237e+09,Put,1806.0
6,6,CON_OP_10001031,10.0,0.1338,0.1356,0.1356,8.0,2958.0,20.96,2.700,...,50ETF沽6月2700A,23.28,0.1401,0.1140,362.0,480766.63,A,1.524237e+09,Put,1806.0
7,7,CON_OP_10001032,2.0,0.1641,0.1674,0.1662,1.0,2253.0,16.66,2.749,...,50ETF沽6月2749A,21.39,0.1728,0.1421,497.0,827421.25,A,1.524237e+09,Put,1806.0
8,8,CON_OP_10001033,1.0,0.1992,0.1951,0.1993,10.0,1346.0,13.23,2.798,...,50ETF沽6月2798A,19.97,0.2073,0.1729,511.0,1028790.92,A,1.524237e+09,Put,1806.0
9,9,CON_OP_10001034,1.0,0.2373,0.2374,0.2394,1.0,1253.0,14.85,2.847,...,50ETF沽6月2847A,17.37,0.2447,0.2088,169.0,397073.43,A,1.524237e+09,Put,1806.0
